In [22]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from snowflake.connector.pandas_tools import write_pandas
import snowflake.connector as sf

In [23]:
url = requests.get("https://ticker.finology.in/")

In [24]:
soup = BeautifulSoup(url.content, 'html.parser')

In [25]:
url.status_code

200

In [26]:
print(soup.prettify())

<!DOCTYPE html>
<html>
 <head>
  <!-- Global site tag (gtag.js) - Google Analytics -->
  <script async="" src="https://www.googletagmanager.com/gtag/js?id=UA-136614031-6">
  </script>
  <script>
   window.dataLayer = window.dataLayer || [];
        function gtag() { dataLayer.push(arguments); }
        gtag('js', new Date());

        gtag('config', 'UA-136614031-6');
  </script>
  <!-- Required meta tags -->
  <title>
   Fundamental Stock Analysis: Balance Sheet, Ratio analysis &amp; financial tools
  </title>
  <meta content="https://ticker.finology.in/" property="og:site_name"/>
  <meta content="Fundamental Stock Analysis: Balance Sheet, Ratio analysis &amp; financial tools" property="og:title"/>
  <meta content="Unlock the Power of Fundamental Stock Analysis: Financial Statements, Balance Sheet, Reports, and Ratios. Free forever." property="og:description"/>
  <meta content="https://ticker.finology.in/images/ticker-link-image-2.jpg" property="og:image"/>
  <meta content="website" p

In [27]:
table = soup.find("table", class_ = "table table-sm table-hover screenertable")

In [28]:
header = table.find_all("th")

In [29]:
header_list = []
for i in header:
    elem = i.text
    header_list.append(elem)
print(header_list)


['Company', 'PriceRs.', 'Day HighRs.']


In [30]:
table_row = table.find_all("tr")

In [31]:
df = pd.DataFrame(columns = header_list)
# print(df)
table_data=[]
for i in table_row[1:]:
    table_data = i.find_all("td")
    table_row = [tr.text for tr in table_data]
    # print(table_row)
    l = len(df)
    df.loc[l] = table_row
print(df)

                     Company  PriceRs. Day HighRs.
0      \nMazagon Dock Ship\n   3566.00     3577.00
1   \nParas Defence &Space\n   1113.00     1130.00
2      \nGarden Reach Ship\n   1638.45     1652.80
3     \nICICI Lombard Gen.\n   1745.25     1766.95
4     \nDixon Technologies\n  11395.40    11458.90
5          \nAmbuja Cement\n    677.10      689.00
6              \nABB India\n   9040.05     9145.00
7    \nGMR Airports Infra.\n     94.09       94.55
8    \nMahindra & Mahindra\n   2942.00     2946.00
9                \nSiemens\n   7855.00     7870.50
10       \nShriram Finance\n   2720.90     2747.00
11                  \nHFCL\n    116.06      118.30
12  \nOracle Finl. Service\n   9574.15     9793.50
13                 \nTrent\n   5169.60     5233.20
14              \nJK Paper\n    487.60      493.00


In [32]:
df = df.map(lambda x: x.strip() if isinstance(x, str) else x)

In [33]:
df

,Company,PriceRs.,Day HighRs.
0,Mazagon Dock Ship,3566.00,3577.00
1,Paras Defence &Space,1113.00,1130.00
2,Garden Reach Ship,1638.45,1652.80
3,ICICI Lombard Gen.,1745.25,1766.95
4,Dixon Technologies,11395.40,11458.90
5,Ambuja Cement,677.10,689.00
6,ABB India,9040.05,9145.00
7,GMR Airports Infra.,94.09,94.55
8,Mahindra & Mahindra,2942.00,2946.00
9,Siemens,7855.00,7870.50


In [34]:
df.to_csv("stock.csv")

In [35]:
connection = sf.connect(user = 'DIVYAASHISHMOHANTY',
                        role = 'ACCOUNTADMIN',
                        password = 'Divyaashish_9438',
                        account = 'ad31709.central-india.azure',
                        warehouse = 'COMPUTE_WH',
                        database = 'PRACTICE', 
                        schema = 'p1')

In [36]:
cursor=connection.cursor()

In [37]:
cursor.execute('use schema p1')

In [38]:
df.columns = map(str.upper,df.columns)

In [39]:
df = df.rename({"COMPANY" : "COMPANY"}, axis = 1)

In [40]:
df = df.rename({"PRICERS." : "PRICE"}, axis=1)

In [41]:
df = df.rename({"DAY HIGHRS." : "DAY_HIGH"}, axis = 1)
df

,COMPANY,PRICE,DAY_HIGH
0,Mazagon Dock Ship,3566.00,3577.00
1,Paras Defence &Space,1113.00,1130.00
2,Garden Reach Ship,1638.45,1652.80
3,ICICI Lombard Gen.,1745.25,1766.95
4,Dixon Technologies,11395.40,11458.90
5,Ambuja Cement,677.10,689.00
6,ABB India,9040.05,9145.00
7,GMR Airports Infra.,94.09,94.55
8,Mahindra & Mahindra,2942.00,2946.00
9,Siemens,7855.00,7870.50


In [42]:
cursor.execute('use schema p1')
write_pandas(connection, df,table_name='STOCK')

C:\Users\KIIT\AppData\Local\Temp\ipykernel_3248\3010490322.py:2: UserWarning: Pandas Dataframe has non-standard index of type <class 'pandas.core.indexes.base.Index'> which will not be written. Consider changing the index to pd.RangeIndex(start=0,...,step=1) or call reset_index() to keep index as column(s)
  write_pandas(connection, df,table_name='STOCK')


(True,
 1,
 15,
 [('dknmskktxq/file0.txt', 'LOADED', 15, 15, 1, 0, None, None, None, None)])